# poetry - python

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[*]").appName('Etanol_Gasolina').getOrCreate()

25/03/07 10:25:45 WARN Utils: Your hostname, SMSSIC10194 resolves to a loopback address: 127.0.1.1; using 172.31.29.9 instead (on interface eth0)
25/03/07 10:25:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 10:25:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/07 10:25:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = (spark
      .read
      .option('delimiter', ';')
      .option('header', 'true')
      .option('inferSchema', 'true')
      .option('enconding', 'ISO-8859-1')
      .csv("./precos-gasolina-etanol-12.csv")
)

In [4]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [5]:
df.createOrReplaceTempView('combustiveis')

In [13]:
spark.sql("""
 select `Estado - Sigla`, `Produto`, `Valor de Compra` , `Valor de Venda` , `Unidade de Medida` from combustiveis;
""").show(50)

+--------------+------------------+---------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+------------------+---------------+--------------+-----------------+
|            SP|          GASOLINA|           NULL|          5,42|       R$ / litro|
|            SP|            ETANOL|           NULL|          3,59|       R$ / litro|
|            AL|          GASOLINA|           NULL|          5,99|       R$ / litro|
|            AP|          GASOLINA|           NULL|          5,83|       R$ / litro|
|            AP|          GASOLINA|           NULL|          5,83|       R$ / litro|
|            AP|          GASOLINA|           NULL|          5,83|       R$ / litro|
|            AP|          GASOLINA|           NULL|          5,89|       R$ / litro|
|            AP|GASOLINA ADITIVADA|           NULL|          6,17|       R$ / litro|
|            AP|          GASOLINA|           NULL|          5,83

In [7]:
spark.sql("""
    select `Estado - Sigla`, `Produto`, `Valor de Compra` , `Valor de Venda` , `Unidade de Medida` from combustiveis where `Valor de Compra` IS NOT NULL;
""").show(5)

+--------------+-------+---------------+--------------+-----------------+
|Estado - Sigla|Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+-------+---------------+--------------+-----------------+
+--------------+-------+---------------+--------------+-----------------+



In [8]:
view_precos = spark.sql("""
    select 
        `Estado - Sigla`, 
        `Produto`, 
        regexp_replace(`Valor de Venda`, ',', '.') as `Valor de Venda`, 
        `Unidade de Medida` 
    from combustiveis;
""")

In [9]:
view_precos.show()

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            SP|          GASOLINA|          5.42|       R$ / litro|
|            SP|            ETANOL|          3.59|       R$ / litro|
|            AL|          GASOLINA|          5.99|       R$ / litro|
|            AP|          GASOLINA|          5.83|       R$ / litro|
|            AP|          GASOLINA|          5.83|       R$ / litro|
|            AP|          GASOLINA|          5.83|       R$ / litro|
|            AP|          GASOLINA|          5.89|       R$ / litro|
|            AP|GASOLINA ADITIVADA|          6.17|       R$ / litro|
|            AP|          GASOLINA|          5.83|       R$ / litro|
|            AP|          GASOLINA|          5.83|       R$ / litro|
|            AP|            ETANOL|          5.11|       R$ / litro|
|            AP|          GASOLINA

In [10]:
view_precos.createOrReplaceTempView('view_precos')

In [11]:
df_diferenca_precos = spark.sql("""
    select 
        `Estado - Sigla`, 
        `Produto`, 
        `Unidade de Medida`,
        MIN(`Valor de Venda`) as menor_valor,
        MAX(`Valor de Venda`) as maior_valor,
        MAX(`Valor de Venda`) - MIN(`Valor de Venda`) as diferenca
    from view_precos
    group by all
    order by diferenca DESC;
""")


In [12]:
df_diferenca_precos.show()

+--------------+------------------+-----------------+-----------+-----------+------------------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|         diferenca|
+--------------+------------------+-----------------+-----------+-----------+------------------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       5.27|       8.89| 3.620000000000001|
|            SP|            ETANOL|       R$ / litro|       3.19|       6.59|               3.4|
|            MG|GASOLINA ADITIVADA|       R$ / litro|       5.67|       8.88| 3.210000000000001|
|            SP|          GASOLINA|       R$ / litro|       4.94|       7.99|              3.05|
|            RJ|          GASOLINA|       R$ / litro|       5.05|       7.79|              2.74|
|            AL|            ETANOL|       R$ / litro|       3.99|       6.49|               2.5|
|            RJ|GASOLINA ADITIVADA|       R$ / litro|       5.49|       7.99|               2.5|
|            SC|            ET